# IMPORT DES LIBRAIRIES

# IMPORT DU DATASET

In [3]:
df = pd.read_csv('comments_students.csv')

KeyboardInterrupt: 

In [3]:
df.head()

,Unnamed: 0,created_utc,ups,subreddit_id,link_id,name,subreddit,id,author,body,parent_id
0,0,1430438400,3.0,t5_2qh1i,t3_34f9rh,t1_cqug90j,AskReddit,cqug90j,jesse9o3,No one has a European accent either because i...,t1_cqug2sr
1,1,1430438400,3.0,t5_2qh1i,t3_34fvry,t1_cqug90k,AskReddit,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry
2,2,1430438400,5.0,t5_2qh1i,t3_34ffo5,t1_cqug90z,AskReddit,cqug90z,InterimFatGuy,NSFL,t1_cqu80zb
3,3,1430438401,1.0,t5_2qh1i,t3_34aqsn,t1_cqug91c,AskReddit,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m
4,4,1430438401,101.0,t5_2qh1i,t3_34f9rh,t1_cqug91e,AskReddit,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc


# TRAVAUX PRELIMINAIRES

### SUPRESSION DES COLONNES INUTILES 

In [4]:
# suppresion de colonnes inexploitable par notre modèle
# subreddit et subreddit id : les individus sont tous identiques ou tous différents
# author : apportera du bruit sur notre modèle si on applique une prédiction avec un author inconnu
df = df.drop(['subreddit', 'subreddit_id', 'author'], axis = 'columns')

In [5]:
df.head()

,Unnamed: 0,created_utc,ups,link_id,name,id,body,parent_id
0,0,1430438400,3.0,t3_34f9rh,t1_cqug90j,cqug90j,No one has a European accent either because i...,t1_cqug2sr
1,1,1430438400,3.0,t3_34fvry,t1_cqug90k,cqug90k,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry
2,2,1430438400,5.0,t3_34ffo5,t1_cqug90z,cqug90z,NSFL,t1_cqu80zb
3,3,1430438401,1.0,t3_34aqsn,t1_cqug91c,cqug91c,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m
4,4,1430438401,101.0,t3_34f9rh,t1_cqug91e,cqug91e,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc


### NETTOYAGE DU CORPUS

In [6]:
# création du data frame rempli avec les corpus de texte
df_txt = pd.DataFrame(columns = ['corpus'])
df_txt['corpus'] = df['body']
df_txt = df_txt.astype({'corpus': str})

In [7]:
# création d'un nouveau dataframe
df_txt_clean = pd.DataFrame()

# supression de la casse
df_txt_clean['corpus'] = df_txt['corpus'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# suppresion des caractère spéciaux
df_txt_clean['corpus'] = df_txt_clean['corpus'].str.replace('[^\w\s]','')

# suppression les stops words
stop = stopwords.words('english')
df_txt_clean['corpus'] = df_txt_clean['corpus'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# suppression des mots communs (on peut changer valeur)
# n'est pas utilisé car on appliquera ce dispositif lors de la création du modèle tf-idf par exemple
"""
#freq = pd.Series(' '.join(df_txt_clean['corpus']).split()).value_counts()[:100]
#freq = list(freq.index)
#df_txt_clean['corpus'] = df_txt_clean['corpus'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
"""

# suppression des mots rares (on peut changer valeur)
# n'est pas utilisé car on appliquera ce dispositif lors de la création du modèle tf-idf par exemple
"""
freq = pd.Series(' '.join(df_txt_clean['corpus']).split()).value_counts()[-10000:]
freq = list(freq.index)
df_txt_clean['corpus'] = df_txt_clean['corpus'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
"""

# lemmatization
df_txt_clean['corpus'] = df_txt_clean['corpus'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

# stemming
# n'est pas utilisé car la lemmatization est plus performant sur notre problème (elle regroupe plus de mots)
"""
st = PorterStemmer()
df_txt_clean['corpus'] = df_txt_clean['corpus'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
"""
df_txt_clean['corpus'].head()

0    one european accent either doesnt exist accent...
1                                kid reminds kevin sad
2                                                 nsfl
3                                im guy idea thing guy
4    mid twenty male rocking skinny jeanspants styl...
Name: corpus, dtype: object

In [ ]:
#df_txt_clean.to_csv('./txt_clean.csv', index=False)
# df_txt_clean = pd.read_csv('txt_clean.csv')
# df_txt_clean = df_txt_clean.astype({'corpus': str})

#### sauvegarde et chargement du texte nettoyé

### CREATION DU RESEAUX NETWORK

In [ ]:
g = nx.DiGraph()
g.add_nodes_from(df.link_id, type = "link")
g.add_nodes_from(df.name, type = "comment")
g.add_edges_from(df[["name", "parent_id"]].values) # , link_type = "parent")
# nx.write_gml(g,"graph.gml")


In [ ]:
# g = nx.read_gml("graph.gml")

# CREATION DES FEATURES

### CREATION DE LA TABLE AVEC FEATURES 

In [11]:
# Création du dataframe qui sera utilisé plus tard pour entraîner le modèle
# df_feat a une colonne 'ups' plus une colonne pour chaque feature
g_nodes = dict(g.in_degree())

# df = pd.DataFrame.from_dict(g_nodes, orient='index')
serie = pd.Series(g_nodes)
 
# sns.distplot(serie, bins = 20, kde=False) # cannot use dataframe with displot
df_feat = pd.DataFrame.from_dict(g_nodes, orient='index')
df_feat = df_feat.reset_index()

# on sélectionne uniquement les noeuds qui sont présents dans le dataset
df_feat = df_feat[df_feat['index'].isin(list(df['name']))] 
df_feat = df_feat.drop('index',axis=1)
df_feat.rename(columns={0: 'entry_nodes'}, inplace=True)
df_feat = df_feat.reset_index(drop=True)
df = df.reset_index(drop=True)
df_feat['ups'] = df['ups']
del df 

## Feature avec corpus

### nombre de mots

In [ ]:
df_txt['word_count'] = df_txt['corpus'].apply(lambda x: len(str(x). split(" ")))
df_feat['word_count'] = list(df_txt['word_count'])
df_txt[['corpus','word_count']].head()

### nombre de charactères

In [ ]:
df_txt['char_count'] = df_txt['corpus'].str.len()
df_feat['char_count'] = list(df_txt['char_count'])
df_txt[['corpus', 'char_count']].head()

### longueur moyenne des mots

In [ ]:
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0 :
        return 1
    else :
        return (sum(len(word) for word in words)/len(words))

df_txt['avg_word'] = df_txt['corpus'].apply(lambda x: avg_word(x))
df_feat['avg_word'] = list(df_txt['avg_word'])
df_txt[['corpus','avg_word']].head()

### nombre de stop words

In [ ]:
stop = stopwords.words('english')

df_txt['stopwords'] = df_txt['corpus'].apply(lambda x: len([x for x in x.split() if x in stop]))
df_feat['stopwords'] = list(df_txt['stopwords'])
df_txt[['corpus','stopwords']].head()

### nombre de charactères numérique

In [ ]:
df_txt['numerics'] = df_txt['corpus'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df_feat['numerics'] = list(df_txt['numerics'])
df_txt[['corpus','numerics']].head()

### nombre de mots en majuscules

In [ ]:
df_txt['upper'] = df_txt['corpus'].apply(lambda x: len([x for x in x.split() if x.isupper() and x != "I"]))
df_feat['upper'] = list(df_txt['upper'])
df_txt[['corpus','upper']].head()

### Sentiments Analysis

In [ ]:
df_txt['sentiment_polarity'] = df_txt_clean['corpus'].apply(lambda x: TextBlob(x).sentiment[0])
df_txt['sentiment_subjectivity'] = df_txt_clean['corpus'].apply(lambda x: TextBlob(x).sentiment[1])
df_txt[['sentiment_polarity', 'sentiment_subjectivity']].head()

In [ ]:
#df_sentiment = df_txt[['sentiment_polarity', 'sentiment_subjectivity']]
#df_sentiment.to_csv('/home/cmisid/Bureau/Scolaire/M1-S2/Projet kaggle/sentiment.csv', index=False)

#df_sentiment = pd.read_csv('/home/cmisid/Bureau/Scolaire/M1-S2/Projet kaggle/sentiment.csv')
#df_txt['sentiment_polarity'] = list(df_sentiment['sentiment_polarity'])
#df_txt['sentiment_subjectivity'] = list(df_sentiment['sentiment_subjectivity'])

In [ ]:
df_feat['sentiment_polarity'] = list(df_txt['sentiment_polarity'])
df_feat['sentiment_subjectivity'] = list(df_txt['sentiment_subjectivity'])

### Word Embedding

In [ ]:
# création du modèle
glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

In [ ]:
# chargement du modèle
filename = 'glove.6B.50d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [ ]:
# tokenisation du corpus afin de pouvoir appliquer le modèle à nos différents mots
sentences = [word_tokenize(s) for s in df_txt_clean['corpus']]

In [ ]:
# création d'un dictionnaire de mots afin d'optimiser les temps de traitement
dict_txt = {}
for i in tqdm_notebook(range(len(df_txt_clean))):
    dict_txt[str(i)] = sentences[i]

In [ ]:
# get_vect
# input : un mot, le modèle
# output : retourne le vecteur du mot ou un vecteur de 0 si le mot n'est pas reconnu dans le modèle
def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return np.zeros(model.vector_size)

# sum_vectors
# input : une phrase, le modèle
# output : retourne la somme de vecteur pour l'ensemble de mots de la phrase
def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

In [ ]:
# création d'un dictionnaire avec les différents vecteurs
dict_vec = {}
for i in tqdm_notebook(range(len(dict_txt))):
    dict_vec[str(i)] = sum_vectors(dict_txt[str(i)], model)

#### sauvegarde et chargement du w2v  

In [ ]:
# sauvegarde du dictionnaire
with open('dict_w2v.csv', 'w', newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for value in tqdm_notebook(dict_vec.values()):
        writer.writerow(value)

In [ ]:
# convertission du dictionnaire en data frame (impossibilité de le faire directement avec panda.DataFrame.from_dic)
w2v = pd.read_csv('dict_w2v.csv', sep = ",", header = -1)

#### w2v kmeans

In [ ]:
# applicatino de kmeans sur les vecteurs
kmeans = KMeans(100).fit(w2v)
df_feat['kmeans_w2v'] = kmeans.labels_

## Features avec graphs

### Degree centrality

In [13]:
degree_c = nx.degree_centrality(g)
df_deg = pd.DataFrame.from_dict(degree_c, orient='index')
df_deg = df_deg.reset_index()
df_deg = df_deg[df_deg['index'].isin(list(df['name']))]
df_feat['degree_c'] = list(df_deg[0])

### Closeness centrality

In [14]:
closeness_c = nx.closeness_centrality(g)
df_deg = pd.DataFrame.from_dict(closeness_c, orient='index')
df_deg = df_deg.reset_index()
df_deg = df_deg[df_deg['index'].isin(list(df['name']))]

In [15]:
df_feat['closeness_c'] = list(df_deg[0])

In [ ]:
df_feat.to_csv("features")

### Granularité

In [16]:
# Ici on calcule le "rang" du commentaire, i.e. 1 si c'est le premier commentaire, 2 si c'est
# un commentaire de commentaire et ainsi de suite. On ajoute 1 si on a pas le lien jusqu'au 
# noeud principal T3.
# Pour cela on utilise la fonction shortest_path_length de network qui calcule les plus petits
# chemins entre un noeud et tous les autres noeuds du réseau. On utilise ensuite le max pour 
# trouver le rang.
list_path_length = []
for i in df.index :
    name = df['name'][i]
    link_id = df['link_id'][i]
    if nx.has_path(g, name, link_id):
        list_path_length.append(max(nx.shortest_path_length(g, name).values()))
    else :
        list_path_length.append(max(nx.shortest_path_length(g, name).values())+1)
df_feat['path_length'] = list_path_length

KeyError: 'link_id'

### Laps de temps entre un commentaire et son parent

In [ ]:
# On utilise un df intermédiaire où on enlève les colonnes innutiles
# On met la colonne 'name' en index, on crée un dictionnaire afin d'optimiser le temps de calcul
df_inter = df.drop(['parent_id','ups','link_id','body','id'], axis = 'columns')
df_inter = df_inter.set_index('name')
dict_name_epoch = df_inter.to_dict()['created_utc']

In [ ]:
list_time_diff = []

# On parcourt la colonne 'name' du dataframe principal. On sélectionne le commentaire parent
# puis on teste si ce parent existe et si ce n'est pas un noeud 't3' car on a pas l'heure des 
# publications. Lorsqu'on a pas les informations on remplit avec np.NaN, sinon on remplit la
# différence entre le commentaire et son parent

for i in df['name']:
    parent_id = list(g.successors(i))
    if parent_id == []: 
        list_time_diff.append(np.NaN)
    elif 't3_' in parent_id[0]: #on exclut les noeuds avec 't3_' car pas dans le dataset
        list_time_diff.append(np.NaN)
    elif parent_id[0] in dict_name_epoch.keys():
        parent_time = dict_name_epoch[parent_id[0]]
        child_time = dict_name_epoch[i]
        list_time_diff.append((child_time - parent_time)/60)
    else : 
        list_time_diff.append(np.NaN)

In [ ]:
df_feat['time_diff'] = list_time_diff

## Aperçu du data frame avec les features

In [ ]:
#df_feat.to_csv('/home/cmisid/Bureau/Scolaire/M1-S2/Projet kaggle/features.csv', index=False)
#df_feat = pd.read_csv('/home/cmisid/Bureau/Scolaire/M1-S2/Projet kaggle/features.csv')

In [ ]:
df_feat.head()

In [ ]:
df_feat.columns

# ENTRAINEMENT DU MODELE 

### CREATION DES DATASETS

In [ ]:
# séparation du jeu de validation
df_val = df_feat[df_feat['ups'].isnull() == True]
df_s = df_feat[df_feat['ups'].isnull() == False]

# séparation des jeux d'entrainement et de tests
y = df_s['ups']
X = df_s.drop('ups', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

### MODELE LGBM

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference = lgb_train)

### PARAMETRISATION DU MODELE & ENTRAINEMENT

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'max_depth' : 7,
    'objective': 'regression_l1',
    'metric': 'l1',
    'num_leaves': 100,
    'learning_rate': 0.01
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round = 500,
                valid_sets = lgb_eval,
                early_stopping_rounds = 10)

### IMPORTANCE DES FEATURES

In [ ]:
lgb.plot_importance(gbm)

### PREDICTION SUR LE TEST

In [ ]:
y_pred = gbm.predict(X_test, num_iteration = gbm.best_iteration)
print('The mae of prediction is :', mean_absolute_error(y_test, y_pred))

### PREDICTION SUR LA VALIDATION

In [ ]:
df_val_y = df_val['ups']
df_val_x = df_val.drop('ups',axis=1)

In [ ]:
y_val = gbm.predict(df_val_x, num_iteration=gbm.best_iteration)

# Création du dataframe et exportation en .csv pour la soumission

In [ ]:
id_val = df[df['ups'].isnull() == True]
id_val = list(id_val['id'])
df_sub = pd.DataFrame({'id':id_val,'predicted':y_val})
df_sub.to_csv('data.csv', index=False)

# Features innutilisées pour la meilleure soumission.

### Time

In [ ]:
# création du data frame en convertissant le created_utc
df_time = pd.DataFrame(columns=['year', 'month', 'day', 'hour'])
for time in tqdm_notebook(df['created_utc']):
    this_time = datetime.utcfromtimestamp(time)
    year = this_time.strftime("%Y")
    month = this_time.strftime("%m")
    day = this_time.strftime("%d")
    hour = this_time.strftime("%H")
    df_time.loc[len(df_time)] = [year, month, day, hour]

# création de la feature hour plus rapide
df_feat['hour'] = df['created_utc'].apply(lambda x: datetime.utcfromtimestamp(x).strftime("%H"))
df_feat['hour'] = pd.to_numeric(df_feat['hour'])

### TF-IDF & K-Means

In [ ]:
# création du tf idf
tfidf = TfidfVectorizer(max_features = 50, analyzer = 'word', ngram_range=(1,2), max_df = 0.80, min_df = 0.04)
df_txt_tfidf = tfidf.fit_transform(df_txt_clean['corpus'])

feature_names = tfidf.get_feature_names()
dense = df_txt_tfidf.todense()
denselist = dense.tolist()
tf_idf = pd.DataFrame(denselist, columns=feature_names)

# Appliquer un k-means sur notre tf-idf
kmeans = KMeans(30).fit(tf_idf)
df_feat['tfidf_kmeans'] = kmeans.labels_

### Prestige degree

In [ ]:
n_nodes = len(g_nodes)
degree_prestige = dict((v,len(g.in_edges(v))/(n_nodes-1)) for v in g.nodes())
df_prestige = pd.DataFrame.from_dict(degree_prestige, orient='index')
df_prestige = df_prestige.reset_index()
df_prestige = df_prestige[df_prestige['index'].isin(list(df['name']))]
df_feat['degree_prestige'] = list(df_prestige[0])

### Upvotes du commentaire parent

In [ ]:
# Création d'un dictionnaire pour interroger les données plus rapidement
# (évite de créer un dataframe avec .iloc() pour une recherche par condition)
# Le dictionnaire a la structure suivante : 
# {'name1' : ups1,
# 'name2' : ups2,
# ...
# }

df_inter = df.drop(['parent_id','created_utc','link_id','body'], axis = 'columns')
df_inter = df_inter.set_index('name')
dict_name_ups = df_inter.to_dict()['ups']
a = []
for i in df['name']:
    parent_id = list(g.successors(i))
    if parent_id == []: 
        a.append(np.NaN)
    elif 't3_' in parent_id[0]: #on exclut les noeuds avec 't3_' car pas dans le dataset
        a.append(np.NaN)
    elif parent_id[0] in dict_name_ups.keys():
        var = dict_name_ups[parent_id[0]]
        a.append(var)
    else : 
        a.append(np.NaN)

df_feat['parent_ups'] = a

### Average neighbor degree

In [ ]:
dict_av_deg = nx.average_neighbor_degree(g)
df_av_ndeg = pd.DataFrame.from_dict(dict_av_deg, orient='index')
df_av_ndeg = df_av_ndeg.reset_index()
df_av_ndeg = df_av_ndeg[df_av_ndeg['index'].isin(list(df['name']))]
df_feat['average_ndeg'] = list(df_av_ndeg[0])

### Moyenne des upvotes par heure (code pas optimisé)

In [ ]:
df['created_utc'] = hour
# on sépare les commentaires et les commentaires avec un 't3_' comme parent_id
mean_per_hour = []
mean_per_hourt3 = []
df_alt_t3 = df[df['parent_id'].str.contains('t3_')]
df_alt_t1 = df[df['parent_id'].str.contains('t1_')]
for i in range (24):
    df_alt = df_alt_t1[df_alt_t1['created_utc']==i]
    df_t3 = df_alt_t3[df_alt_t3['created_utc']==i]
    mean_per_hourt3.append(np.mean(df_t3['ups']))
    mean_per_hour.append(np.mean(df_alt['ups']))
tab_mean = []
for i in range(len(df)):
    for k in range(24):
        if df['created_utc'][i] == k :
            if 't3_' in df['parent_id'][i] :
                tab_mean.append(mean_per_hourt3[k])
            else :
                tab_mean.append(mean_per_hour[k])
df_feat['mean_per_hour'] = tab_mean

# Implémentation d'une GridSearch

In [ ]:
mdl = lgb.LGBMRegressor(boosting_type= 'gbdt', 
          objective = 'regression_l1', 
          max_depth = 5,
    metric = 'l1',
    num_leaves = 31,
    learning_rate = 0.05
          )
gridParams = {
    'learning_rate': [0.001],
    'n_estimators': [100],
    'num_leaves': [31,100,250], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt'],
    'bagging_fraction' : [0.2,0.5,0.8],
    'min_data_in_leaf' : [200,600,1000]
}

grid = GridSearchCV(mdl, gridParams)
# Run the grid
grid.fit(X, y)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)